<a href="https://colab.research.google.com/github/Kamaljit12/Brain-tumor-Prediction-CNN/blob/main/Analysis_started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list -s 'brain tumor datasets'

ref                                                               title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
davidbroberts/brain-tumor-object-detection-datasets               Brain tumor object detection datasets                12MB  2021-10-04 02:36:32           3429         49  0.875            
masoudnickparvar/brain-tumor-mri-dataset                          Brain Tumor MRI Dataset                             149MB  2021-09-24 12:43:45          66994        632  0.875            
trainingdatapro/computed-tomography-ct-of-the-brain               Computed Tomography (CT) of the Brain                66MB  2023-10-13 15:17:47           3415         55  1.0              
awsaf49/brats20-dataset-training-validation       

In [ ]:
# download datasets
!kaggle datasets download -d 'masoudnickparvar/brain-tumor-mri-dataset'

Dataset URL: https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset
License(s): CC0-1.0
 83% 124M/149M [00:00<00:00, 182MB/s]
100% 149M/149M [00:00<00:00, 189MB/s]


In [ ]:
## unzip
import zipfile
zip_ref = zipfile.ZipFile('/content/brain-tumor-mri-dataset.zip', 'r')
zip_ref.extractall('/content/data')
zip_ref.close()

In [ ]:
## load this image datasets
import tensorflow as tf

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory('/content/data/Training',
                                                       image_size=(256, 256))

val_ds = tf.keras.utils.image_dataset_from_directory('/content/data/Testing',
                                                     image_size=(256, 256))

Found 5712 files belonging to 4 classes.
Found 1311 files belonging to 4 classes.


In [ ]:
len(train_ds), len(val_ds)

(179, 41)

In [ ]:
labels = train_ds.class_names
print(labels)

['glioma', 'meningioma', 'notumor', 'pituitary']


In [ ]:
# train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
# val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
labels = []
for image, label in train_ds.as_numpy_iterator():
  labels.append(label)

In [ ]:
ls = []
for i in labels:
  for j in i:
    ls.append(j)

In [ ]:
import numpy as np
import pandas as pd

arr = np.array(ls)
pd.DataFrame(arr).value_counts()

## there is no imballencing

,count
0,
2,1595
3,1457
1,1339
0,1321


In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(ncols=4,figsize=(10, 4))

batch = train_ds.shuffle(5).as_numpy_iterator().next()

for index, image in enumerate(batch[0][:4]):
  ax[index].imshow(image.astype(int))
  ax[index].title.set_text(batch[1][index])
plt.show()

In [ ]:
def process(image, label):
  image = tf.image.resize(image, (256, 256))
  return image, label

train_ds = train_ds.map(process)
val_ds = val_ds.map(process)

In [ ]:
num_of_class = len(np.unique(train_ds.as_numpy_iterator().next()[1]))

In [ ]:
## trian the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization

model = Sequential([
    Conv2D(32, kernel_size=(3,3), strides=2, activation='relu', padding='valid',input_shape=(256, 256, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.4),

    Conv2D(64, (3,3), strides=2, padding='valid',activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.4),

    Conv2D(128, (3,3), strides=2, padding='valid',activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.4),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.4),
    Dense(32, activation='relu'),
    Dropout(0.4),
    Dense(num_of_class, activation='softmax')

])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)                   │ (None, 127, 127, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_12               │ (None, 127, 127, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_21 (Dropout)                 │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 31, 31, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_13               │ (None, 31, 31, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 15, 15, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_22 (Dropout)                 │ (None, 15, 15, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 7, 7, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_14               │ (None, 7, 7, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_14 (MaxPooling2D)      │ (None, 3, 3, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_23 (Dropout)                 │ (None, 3, 3, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 1152)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 128)                 │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_24 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_25 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_26 (Dropout)                 │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 4)                   │             1

 Total params: 252,196 (985.14 KB)

 Trainable params: 251,748 (983.39 KB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
## fit and train the model
history = model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 28s 115ms/step - accuracy: 0.3330 - loss: 1.6298 - val_accuracy: 0.5400 - val_loss: 1.1455
Epoch 2/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 28s 75ms/step - accuracy: 0.5711 - loss: 1.0371 - val_accuracy: 0.6194 - val_loss: 0.9655
Epoch 3/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 20s 72ms/step - accuracy: 0.6559 - loss: 0.8793 - val_accuracy: 0.6339 - val_loss: 0.9802
Epoch 4/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 20s 68ms/step - accuracy: 0.7159 - loss: 0.7543 - val_accuracy: 0.6606 - val_loss: 0.8428
Epoch 5/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 13s 73ms/step - accuracy: 0.7319 - loss: 0.7018 - val_accuracy: 0.7132 - val_loss: 0.6426
Epoch 6/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 20s 69ms/step - accuracy: 0.7515 - loss: 0.6526 - val_accuracy: 0.5744 - val_loss: 1.8263
Epoch 7/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 21s 69ms/step - accuracy: 0.7910 - loss: 0.6255 - val_accuracy: 0.6728 - val_loss: 1.0768
Epoch 8/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 13s 70ms/step - accuracy: 0.8016 - loss: 0.5654 -

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:
## test the model
test_ds = tf.keras.utils.image_dataset_from_directory('/content/data/Testing',
                                                       image_size=(256, 256))

Found 1311 files belonging to 4 classes.


In [ ]:
batch = test_ds.as_numpy_iterator().next()

In [ ]:
img = batch[0][1]
img_label = batch[1][1]
plt.imshow(img.astype(int))
plt.show()

In [ ]:
image = tf.image.resize(img, (256, 256))
image = tf.expand_dims(image, 0)

In [ ]:
labels = ['glioma', 'meningioma', 'notumor', 'pituitary']

In [ ]:
prediction = model.predict(image)
index = np.argmax(prediction)
print(f"Predicted: {labels[index]}")
print(f"Actual: {labels[img_label]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Predicted: glioma
Actual: glioma
